# RAG over complex PDF


## Setting
 - Auto Reload
 - path for utils

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
module_path = "../../.."
sys.path.append(os.path.abspath(module_path))

In [5]:
!pip install -U lxml
!pip install -U pydantic
!pip install -U kaleido
!pip install -U uvicorn
!pip install -U pandas
!pip install -U numexpr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.6.3
    Uninstalling pydantic-2.6.3:
      Successfully uninstalled pydantic-2.6.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: uvicorn
    Found existing installation: uvicorn 0.27.1
    Uninstalling uvicorn-0.27.1:
      Successfully uninstalled uvicorn-0.27.1


In [ ]:
!sudo rpm -Uvh https://dl.fedoraproject.org/pub/epel/epel-release-latest-7.noarch.rpm
!sudo yum -y update
!sudo yum install -y poppler-utils

설치: https://gist.github.com/codesankalp/c4d8e18421bf112a70406241f6d91ee9

In [ ]:
!sudo yum install clang -y && \
 sudo yum install libpng-devel libtiff-devel zlib-devel libwebp-devel libjpeg-turbo-devel wget tar gzip -y && \
 wget https://github.com/DanBloomberg/leptonica/releases/download/1.75.1/leptonica-1.75.1.tar.gz && \
 tar -zxvf leptonica-1.75.1.tar.gz && \
 cd leptonica-1.75.1 && \
 ./configure && \
 sudo make && \
 sudo make install
#cd ~ && \
!sudo yum install git-core libtool pkgconfig -y && \
 git clone --depth 1  https://github.com/tesseract-ocr/tesseract.git tesseract-ocr && \
 cd tesseract-ocr && \
 export PKG_CONFIG_PATH=/usr/local/lib/pkgconfig && \
 ./autogen.sh && \
 ./configure && \
 sudo make && \
 sudo make install

!cd /usr/local/share/tessdata && \
 sudo wget https://github.com/tesseract-ocr/tessdata/raw/main/osd.traineddata && \
 sudo wget https://github.com/tesseract-ocr/tessdata/raw/main/eng.traineddata && \
 sudo wget https://github.com/tesseract-ocr/tessdata/raw/main/hin.traineddata

!sudo rm -rf leptonica-1.75.1 leptonica-1.75.1.tar.gz tesseract-ocr

In [ ]:
!pip install "unstructured[all-docs]"

amazon linux tesseract 설치: https://blog.hikmahtechnologies.com/installing-tesseract-on-amazon-linux-2-4a08517eb2f1

소스 코드: https://github.com/sudarshan-koirala/youtube-stuffs/blob/main/langchain/langchain_Semi_Structured_RAG.ipynb

In [ ]:
import urllib.request

url = "https://arxiv.org/pdf/2307.09288.pdf"
filename = "Llama2.pdf"
urllib.request.urlretrieve(url, filename)

In [3]:

path = "./content/"

In [4]:

from lxml import html
from pydantic import BaseModel
from typing import Any, Optional
from unstructured.partition.pdf import partition_pdf

# Get elements
raw_pdf_elements = partition_pdf(
    filename=path+"dart-short-sample.pdf",
    # Unstructured first finds embedded image blocks
    extract_images_in_pdf=True,
    # Use layout model (YOLOX) to get bounding boxes (for tables) and find titles
    # Titles are any sub-section of the document
    infer_table_structure=True,
    # Post processing to aggregate text once we have the title
    chunking_strategy="by_title",
    # Chunking params to aggregate text blocks
    # Attempt to create a new chunk 3800 chars
    # Attempt to keep chunks > 2000 chars
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    #image_output_dir_path=path
)

Ignoring (part of) ToUnicode map because the PDF data does not conform to the format. This could result in (cid) values in the output. The start object is not a byte.
Ignoring (part of) ToUnicode map because the PDF data does not conform to the format. This could result in (cid) values in the output. The start object is not a byte.
This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name
Ignoring (part of) ToUnicode map because the PDF data does not conform to the format. This could result in (cid) values in the output. The start object is not a byte.
Ignoring (part of) ToUnicode map because the PDF data does not conform to the format. This could result in (cid) values in the output. The start object is not a byte.
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: [

In [5]:
# Create a dictionary to store counts of each type
category_counts = {}

for element in raw_pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.CompositeElement'>": 21,
 "<class 'unstructured.documents.elements.Table'>": 19}

In [6]:

class Element(BaseModel):
    type: str
    text: Any

# Categorize by type
categorized_elements = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        categorized_elements.append(Element(type="table", text=str(element)))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        categorized_elements.append(Element(type="text", text=str(element)))

# Tables
table_elements = [e for e in categorized_elements if e.type == "table"]
print(len(table_elements))

# Text
text_elements = [e for e in categorized_elements if e.type == "text"]
print(len(text_elements))

19
21


In [11]:
for e in categorized_elements:
    print (e.type)
    print (e)
    print ("==")

text
type='text' text='[oj2HosiE )\n\n)\n\n+2IZR(X])-2021-316\n\n=L =| IT A ZSSH(ELS) HHU(QA)SELHM\n\n=L A ZSSH(ELS)\n\n=|\n\n=|\n\nIT\n\n@ 0] M= S8AH|KI| #e| HS I SZYEM Higt Ofs STIS #i510] "FSLHIX HS0]| 2ot HE, R 2 70 27, GALe| LT SHIEAIS 7HA MUZESHEO| =0 LHZS 2l Olohet = AR Zhyet XIZRLICE lo\n\n70\n\n2\n\nR\n\nHE,\n\n2ot\n\nHS0]|\n\n"FSLHIX\n\n#i510]\n\nSTIS lo\n\nOfs\n\n27,\n\nHigt\n\nS8AH|KI|\n\n@ 0]\n\n0]\n\nI\n\nHS I\n\nM=\n\n#e|\n\nSZYEM\n\nGALe|\n\nLT\n\nSHIEAIS\n\nXIZRLICE\n\n=0\n\nAR\n\n7HA\n\n=\n\nOlohet\n\nMUZESHEO|\n\nLHZS\n\n=0\n\nZhyet\n\n2l\n\n® 0] MBME THZFSHEO| EXLITM(RHEAIRT SSEAN Bet UE 0 2H)2 HES BORSIAZH DHEMH Oty S0F S MHE ST UYSLICL MR, THLUMME 0] MM} &7 IZHZHO| SXMIME YA o} YEE OlaHStAI=Tl ISHAI7| HHELICE\n\nDHEMH\n\nBORSIAZH\n\nHES\n\nUE 0\n\nBet\n\nSSEAN\n\nOty\n\n® 0]\n\n0]\n\nMBME\n\nTHZFSHEO|\n\nYA\n\nSXMIME\n\nS0F\n\nS\n\nIZHZHO|\n\n&7\n\nUYSLICL\n\nMHE\n\n0]\n\nTHLUMME\n\no}\n\nST\n\nMR,\n\nMM}\n\nOlaHStAI=Tl\n\nYEE\n\nHHELICE\n\n@ 2 482 2 (iZYEY 22 Y9 

## 1. Bedrock Client 생성

In [12]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [13]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Claude-V3-Sonnet': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


## 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

### LLM 로딩 (Claude-v2)

In [14]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [15]:
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2-1"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 1024
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

llm_text

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7ff4985d4b50>, model_id='anthropic.claude-v2:1', model_kwargs={'max_tokens_to_sample': 1024}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7ff49859b5b0>])

In [16]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

In [17]:

# Prompt
prompt_text="""You are an assistant tasked with summarizing tables and text. \
Give a concise summary of the table or text. Table or text chunk: {element} """
prompt = ChatPromptTemplate.from_template(prompt_text)

# Summary chain
#model = ChatOpenAI(temperature=0,model="gpt-3.5-turbo")

summarize_chain = {"element": lambda x:x} | prompt | llm_text | StrOutputParser()

In [18]:

# Apply to tables
tables = [i.text for i in table_elements]
table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

 Unfortunately I do not have access to the table or text chunk that was referenced. As an AI assistant without access to additional context, I am unable to summarize something that was not provided. If a table, text excerpt, or other content is provided for me to summarize, I would be happy to attempt to concisely summarize its key information. Please feel free to provide a table, text, or other content you would like me to summarize. 이 테이블은 구조화된 금융상품의 위험등급에 대한 기준을 보여줍니다. 1등급부터 5등급까지 초고위험에서 초저위험 수준으로 분류합니다. 각 위험등급별로 원금비보장형 ELS, DLS의 녹인배리어 범위, Pay off 레버리지 원금 지급 기준 등을 제시하고 있습니다. Unfortunately I do not have access to the table you referenced. As an AI assistant without visual perception capabilities, I need the textual content of tables or images provided explicitly in order to summarize them. If you provide the textual contents, I would be happy to generate a concise summary. Please feel free to provide the text content of the table, and I will do my best to summarize it helpfully. 이 테이

In [19]:
table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

 Unfortunately I do not have access to the table or text chunk that was referenced. As an AI assistant without access to additional context, I am unable to summarize content that has not been provided. If a table, text excerpt, or other content is provided for me to summarize, I would be happy to attempt to generate a concise summary of it. Please feel free to provide me with the information you would like summarized and I will do my best to produce a useful summary. Let me know if you need any clarification or have additional requests! 이 테이블은 구조화금융상품(ELS, DLS)의 신용등급을 1등급(초고위험)부터 5등급(초저위험)까지 5개 등급으로 분류하고, 각 등급별로 원금비보장형과 지급형 상품의 기준을 제시하고 있습니다. 원금비보장형은 녹인배리어와 페이오프 레버리지 정도에 따라, 지급형은 원금지급 범위에 따라 등급이 구분됩니다. Unfortunately I do not have access to the table you referenced. As an AI assistant without access to additional context, I am unable to summarize a table or text that was not provided. Please provide the full table or text you would like me to summarize. I'd be happy to condense the key 

### Embedding 모델 선택

In [ ]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

In [ ]:
def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):

    if is_bedrock_embeddings:
        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings
        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
            )
        )
        print("Bedrock Embeddings Model Loaded")

    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")

    return llm_emb

#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [ ]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)   

## 3. LangChainOpenSearch VectorStore 정의
### 선수 조건
- 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 OpenSearch Index 가 생성이 되어 있어야 합니다.
#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.
- 01_preprocess_docs/01_parameter_store_example.ipynb 참고

In [ ]:
import boto3
from utils.ssm import parameter_store

In [ ]:
region=boto3.Session().region_name
pm = parameter_store(region)

In [ ]:
opensearch_domain_endpoint = pm.get_params(
    key="opensearch_domain_endpoint",
    enc=False
)

opensearch_user_id = pm.get_params(
    key="opensearch_user_id",
    enc=False
)

opensearch_user_password = pm.get_params(
    key="opensearch_user_password",
    enc=True
)

In [ ]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

### Index 이름 셋팅
- 이전 노트북 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 생성된 OpenSearch Index name 입력

In [ ]:
index_name = opensearch_user_password = pm.get_params(
    key="opensearch_index_name",
    enc=True
)

print (f'index_name: {index_name}')

### OpenSearch Client 생성

In [ ]:
from utils.opensearch import opensearch_utils

In [ ]:
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 4. Retriever based on Hybrid Search 정의
- LangChain에서 제공하는 **BaseRetriever** 클래스를 상속받아 **Custom Retriever**를 정의 할 수 있습니다.
- 본 샘플코드 에서는 **Hybrid Search based Retriever**를 **정의**합니다. 

OpenSearch Hybrid 는 아래와 같은 방식으로 작동합니다.
- (1) Sematic serch를 통해 각 document별 relevant score 산출
- (2) Lexical search를 통해 각 document별 relevant score 산출
- (3-1) Rank-fusion 방식이 "simple weighted" 일 경우
    - 산출된 score에 대한 normalization 수행
    - 전체 결과에서 가장 높은 스코어는 표준화 과정을 통하여 스코어가 1.0 이 됨.
- (3-2) Rank-fusion 방식이 "Reciprocal Rank Fusion (RRF)" 일 경우
    - Paper: https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
    - Desc: https://medium.com/@sowmiyajaganathan/hybrid-search-with-re-ranking-ff120c8a426d
    - **RRF의 경우 score가 아닌 ranking 정보를 활용, 때문에 score normalization이 필요 없음**
    - ![rrf.png](../../../10_advanced_question_answering/img/rrf.png)

RRF는 langchain에서 "Ensemble Retriever" 이름으로 api를 제공합니다. 
- https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble

In [ ]:
from utils.rag import OpenSearchHybridSearchRetriever

- 필터 설정 예시
- filter=[ <BR>
    　{"term": {"metadata.[**your_metadata_attribute_name**]": "**your first keyword**"}}, <BR>
    　{"term": {"metadata.[**your_metadata_attribute_name**]": "**your second keyword**"}},<BR>
]

In [ ]:
opensearch_hybrid_retriever = OpenSearchHybridSearchRetriever(
    # necessary
    os_client=os_client,
    index_name=index_name,
    llm_emb=llm_emb,
    llm_text=llm_text,

    # option for lexical
    minimum_should_match=0,
    filter=[],

    # option for rank fusion
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"], rank fusion 방식 정의
    ensemble_weights=[.51, .49], # [for lexical, for semantic], Lexical, Semantic search 결과에 대한 최종 반영 비율 정의

    # option for async search
    async_mode=True,

    # option for output
    k=5, # 최종 Document 수 정의
    verbose=True,
)

### Retrieval example
- default search

In [ ]:
from utils.rag import show_context_used

In [ ]:
#query = "중지된 경우 이체"
query = "vidio max size?"

In [ ]:
%%time
search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents(query)

print("\n==========  Results  ==========\n")
print(f'1. question: {query}')
print (f'2. # documents: {len(search_hybrid_result)}')
print("3. Documents: \n")

show_context_used(search_hybrid_result)

- update parameters

In [ ]:
opensearch_hybrid_retriever.update_search_params(
    k=10,
    minimum_should_match=0,
    filter=[],
    #filter=[
    #    {"term": {"metadata.seq_num": "64"}},
    #],
    async_mode=True
)

In [ ]:
#query = "중지된 경우 이체"
query = "vidio max size?"
search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents(query)

print("\n==========  Results  ==========\n")
print(f'1. question: {query}')
print(f'2. # documents: {len(search_hybrid_result)}')
print("3. Documents: \n")

show_context_used(search_hybrid_result)

## 5. RAG using RetrievalQA powered by LangChain

In [ ]:
from utils.rag import prompt_repo
from langchain.prompts import PromptTemplate

### Prompting
- [TIP] Prompt의 instruction의 경우 한글보다 영어로 했을 때 더 좋은 결과를 얻을 수 있습니다.

In [ ]:
PROMPT = prompt_repo.get_qa(prompt_type="answer_only") # ["answer_only", "answer_with_ref"]
pprint (PROMPT.template)

### Update Search Params (Optional)

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
opensearch_hybrid_retriever.update_search_params(
    k=5,
    minimum_should_match=3,
    #filter=[
    #    {"term": {"metadata.project": "KS"}},
    #],
    filter=[],
    async_mode=True
)

### Request

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm_text,
    chain_type="stuff",
    retriever=opensearch_hybrid_retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": PROMPT,
        "verbose": False,
    },
    verbose=False
)

In [ ]:
#query = "중지된 경우 이체"
query = "vidio max size?"
response = qa(query)

In [ ]:
print("##################################")
print("query: ", query)
print("##################################")

print (colored("\n\n### Answer ###", "blue"))
print_ww(response['result'])

print (colored("\n\n### Contexts ###", "green"))
show_context_used(response['source_documents'])